## Calibration-Less Multi-coil (CaLM) MRI
Angshul Majumdar and Rabab K.Ward, 2012

### Assumption
- Sensitivity encoded images of the different coils are consistent with each other
- Image is sparse in some basis
- Measurement matrix has RIP


### Image reconstruction
The data aquistion can be expressed as s 

$$y_i = PFS_ix + \eta_i,  \quad i = 1,\dots,C \qquad (1)$$

where 
- $x \in \mathbb{R}^n$ is the vectorized image
- $S_i$ is the diagonalized smooth sensitivity map of the $i$-th coil-
- $P$ is a fat $m \times n, \: m < n $ permuation matrix for undersamlpling the k-space (each row in P has only one $1$ and rest are zeros)
- $y_i \in \mathbb{C}^m $ is the undersampled k-space data of $i$-th coil
- $\eta$ denotes the noise.

Denoting the sensitivity encoded images by $x_i = S_i x$, we further reformulate above equation as 

$$y_i = PFx_i + \eta_i,  \quad i = 1,\dots,C \qquad (2)$$

Now, employing the wavelet transform on the sensitivity encoded images using the transformation matrix $W$, we obtain

$$y_i = PFW^T\alpha_i + \eta_i,  \quad i = 1,\dots,C \qquad (3)$$

with wavelet coefficents $\alpha_i$.
Alternatively, by concatinating all $y_i$ into one large measurement vector $y = \begin{bmatrix}y_1\\ \vdots \\ y_C\end{bmatrix}$ yields

$$ y = A\alpha +  \eta, \qquad (4)$$

where $A = \text{diag}\bigg(\begin{bmatrix} PFW^T \\ \vdots \\ PFW^T \end{bmatrix}\bigg)$ is the measurement matrix, $\alpha = \begin{bmatrix}\alpha_1\\ \vdots \\ \alpha_C\end{bmatrix}$, and $\eta = \begin{bmatrix}\eta_1\\ \vdots \\ \eta_C\end{bmatrix}$.

The task is now to solve (4) for the coefficiens $\alpha$. Since (4) is an undertermined equations system, we employ the theories of compressive sensing to reconstructed a unique solution. That is, we assume 

1) the coefficient vector $\alpha$ is sparse,

2) the measurement matrix $A$ satisfies the restricted isometric property (RIP).

Typically for MRI images, their wavelet coeffiecents are sparse, thus assuming that $\alpha$ is sparse is a reasonable assumption. Furhermore, we do not only assume $\alpha$ is sparse but rather that it is _group-sparse_. That means the different $\alpha_i$ share the approximatelty the same support, but not necessarily the same values at the non-zero entries. This follows from the assumption that the sensitivity maps $S_i$ are smooth and therefore the multiplication $x_i = S_i x$ will yield discontinuities at the same positions as the image $x$. And as the wavelet transform encodes the discontinuities we can expect $\alpha$ to be group-sparse.

Remark: The authors for the CaLM MRI paper claim that also the values of the coefficents at the same indices in different $\alpha_i$s will be similar; however, I cannot comprehend their reasoning as I think that I could find a smooth sensitivity map to show that it is not true.

Denoting the matrix $\alpha_S = \begin{bmatrix}\alpha_1 & \cdots & \alpha_C\end{bmatrix}$ the optimization to obtain $\alpha$ becomes

$$ \hat{\alpha}_S = \text{argmin}_{\alpha_S}\: ||\alpha_S||_{2,1}, \quad \text{s.t.} \: ||y-A\alpha||_2 \leq \epsilon \qquad(5)$$

where $\epsilon = \sqrt{CN\sigma}$ and $||\cdot||_{2,1}$ denotes the $\ell_{2,1}$ norm.

Formulation (5) is a synthesis prior formulation; however, a solution can also be obtained by solving the analysis prior formulation

$$ \hat{X} = \text{argmin}_{X} \: ||WX||_{2,1}, \quad \text{s.t.} \: ||y-\text{vec}(PFX)||_2 \leq \epsilon, \qquad(6)$$

where $X = \begin{bmatrix}x_1 & \cdots & x_C\end{bmatrix}$. The analysis prior formulation thus solves for the sensitivity encoded images directly. Since $W$ is a orthogonal matrix, (5) and (6) yield the same solution in theory. 

Upon obtaining the sensitivity encoded images, the final image $\hat{x}$ is computed as the sum of squares, i.e, $\hat{x} = \sqrt{\sum_{i = 1}^C \hat{x}_i^2}$.


### Sampling Approaches

Periodic:
- traditionally the most often used parallel MRI acceleration method
- unsuitable for CS reconstruction

Gaussian:
- yields good results
- can not be efficiently acquired

Radial:
- practical and efficient for CS based MR
- robust to object motion and ghosting artifcts
- reconstructing images more difficult 

<img src="./pics/calm_sampl.png">

### Similarity and differences to GRAPPA and SENSE  

Similiarities:

- Mulit-coil
- Produces one sensitivity encoded image for each coil, like GRAPPA
- Formalism like SENSE

Differences:

- No calibration needed, no parameter estimation (sensitivity maps in SENSE or interpolation weights in GRAPPA)
- Insensitive to calibration errors
- Unlike SENSE, one image each coil
- Underdetermined equation system

Negatves:

- Can lead to oversmooth, blurred images
- Depended on the sampling approach and sparsefying transform

#### My Criticism

- Consitency assumptions of sensitivity encoded images of the different coils
- SOS of sensitivity encoded images
- Is group sparsity really needed
- No really effective usage of multi coil setting




<img src="pics/img_for_pres.png">

From paper: Reconstructed images from Gaussian Sampling. Left to right: brain, Shepp-Logan Phantom and UBC MRI Slice 1; top to bottom: GRAPPA, CS SENSE, CaLM MRI synthesis, CaLM MRI analysis.

### Low-Rank Formulation for CaLM MRI
An alternative formulation of CaLM MRI is its low rank formulation. That is, we assume $X$ to be low rank and solve

$$ \hat{X} = \text{argmin}_X \: ||Y −PFX||_F$$

where $Y = \begin{bmatrix}y_1 & \cdots & y_C\end{bmatrix}$, and X is constrained be low rank. The low rank assumption comes from the observation that neighbouring coils yield approximately linearly correlated images.